In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
# Simulation
import logging
import sys

import arrow
import matplotlib.pyplot as plt
import pandas as pd
import simpy

from src.machine import *
from src.operator import *
from src.utils import *
from src.base import *
from src.material import *
from src.schedule import *
from src.product import *
from src.consumable import *
from src.bom import BOM


logging.basicConfig(
     stream=sys.stdout,
     level=logging.DEBUG,
     format='%(asctime)s - %(name)s - %(levelname)-7s - %(message)s',
     datefmt='%H:%M:%S'
 )
logger = logging.getLogger(__name__)
logger.info('Starting simulation')


start = arrow.now('Europe/Helsinki')
env = simpy.Environment(initial_time=start.timestamp())

01:36:03 - __main__ - INFO    - Starting simulation


In [2]:
import yaml
from copy import deepcopy


with open('config/factory.yml', 'r') as f:
    cfg = yaml.full_load(f.read())


def cfg2obj(env, obj, cfg_list):
    cfg_list = deepcopy(cfg_list)
    out = {}
    for cfg in cfg_list:
        id_ = cfg.pop('id')
        out[id_] = obj(env, **cfg)
    return out


def make_boms(env, cfg_list, material_objs, consumable_objs, product_objs):
    cfg_list = deepcopy(cfg_list)
    out = {}
    for cfg in cfg_list:
        id_ = cfg.pop('id')

        # Materials
        materials_ = {}
        for material in cfg.pop('materials', {}):
            obj = material_objs[material['id']]
            # TODO: Consume time as well?
            consumption = float(material['consumption']) / 60 / 60
            materials_[obj] = {'consumption': consumption}

        # Consumables
        consumables_ = {}
        for consumable in cfg.pop('consumables', {}):
            obj = consumable_objs[consumable['id']]
            consumption = float(consumable['consumption']) / 60 / 60
            consumables_[obj] = {'consumption': consumption}

        # Products (output)
        products_ = {}
        for product in cfg.pop('products', {}):
            obj = product_objs[product['id']]
            quantity = float(product['quantity'])
            products_[obj] = {'quantity': quantity}

        out[id_] = BOM(env, **cfg,
                       materials=materials_,
                       consumables=consumables_,
                       products=products_)

    return out


def make_programs(env, cfg_list, boms):
    cfg_list = deepcopy(cfg_list)
    out = {}
    for cfg in cfg_list:
        id_ = cfg.pop('id')
        bom_ = boms[cfg.pop('bom')]
        out[id_] = Program(env, bom=bom_, **cfg)

    return out


def make_schedules(env, cfg_list, programs):
    cfg_list = deepcopy(cfg_list)
    out = {}
    for cfg in cfg_list:
        id_ = cfg.pop('id')
        blocks_ = []
        for block in cfg.pop('blocks', []):
            # TODO: Support others than just cron
            block_obj = CronBlock(
                env,
                start_expr=block['cron'],
                duration_hours=block['duration-hours'],
                program=programs[block['program']]
            )
            blocks_.append(block_obj)
        out[id_] = OperatingSchedule(env, blocks=blocks_, **cfg)

    return out


def make_machines(env, cfg_list, programs, schedules):
    cfg_list = deepcopy(cfg_list)
    out = {}
    for cfg in cfg_list:
        d = {}
        id_ = cfg.pop('id')
        if 'name' in cfg:
            d['name'] = cfg['name']
        if 'programs' in cfg:
            d['programs'] = [programs[program_id] for program_id in cfg['programs']]
        if 'schedule' in cfg:
            d['schedule'] = schedules[cfg['schedule']]
        if 'default-program' in cfg:
            d['default_program'] = programs[cfg['default-program']]

        out[id_] = Machine(env, **d)

    return out
        


materials = cfg2obj(env, Material, cfg['materials'])
consumables = cfg2obj(env, Consumable, cfg['consumables'])
products = cfg2obj(env, Product, cfg['products'])
boms = make_boms(env, cfg['boms'], materials, consumables, products)
programs = make_programs(env, cfg['programs'], boms)
schedules = make_schedules(env, cfg['schedules'], programs)
machines = make_machines(env, cfg['machines'], programs, schedules)
operator = Operator(env).assign_machine(machines['machine1'])

In [3]:
env.run(until=start.timestamp() + 1 * 24 * 60 * 60)

01:36:03 - src.base - INFO    - 2022-12-02 01:36:03 - Cron(30 8 * * *, 2h, Program(My program 1)) - Cron scheduled for 2022-12-02 08:30:00 - 2022-12-02 10:29:59
01:36:03 - src.base - INFO    - 2022-12-02 01:36:03 - Cron(00 12 * * *, 1.5h, Program(My program 2)) - Cron scheduled for 2022-12-02 12:00:00 - 2022-12-02 13:29:59
01:36:03 - src.base - INFO    - 2022-12-02 01:36:03 - Operator - Chilling at home...
01:36:03 - src.base - DEBUG   - 2022-12-02 01:36:03 - OperatingSchedule(Basic schedule) - Event - "machine_assigned"
01:36:03 - src.base - INFO    - 2022-12-02 08:00:00 - Operator - Working...
01:36:03 - src.base - DEBUG   - 2022-12-02 08:00:00 - Operator - Event - "arrive_at_work"
01:36:03 - src.base - DEBUG   - 2022-12-02 08:00:00 - Operator - Waiting for issues
01:36:03 - src.base - DEBUG   - 2022-12-02 08:00:01 - Machine(My machine) - Event - "on_button_pressed"
01:36:03 - src.base - DEBUG   - 2022-12-02 08:00:02 - Machine(My machine) - Event - "switching_on"
01:36:03 - src.base 

In [4]:
import plotly.express as px


def plot_timeline(data, **kwargs):
    df = (
        pd.DataFrame(data, columns=['ds', 'obj', 'key', 'value'])
        .assign(name=lambda df_: df_['obj'] + ' - ' + df_['key'],
                end_ts=lambda df_: df_.groupby(['obj', 'key'])['ds'].shift(-1).fillna(machine.now_dt.datetime),
                value=lambda df_: df_['value'].astype(str))
    )
    if df.empty:
        return

    fig = px.timeline(df, facet_row='name', x_start='ds', x_end='end_ts', y='value',
                      title='Simulation - Categorical', color='obj', **kwargs)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split(' - ')[-1]))
    fig.update_yaxes(matches=None)
    fig.show()


def plot_numerical(data, **kwargs):

    df = pd.DataFrame(data, columns=['ds', 'obj', 'key', 'value'])
    df['end_ts'] = df['ds'].shift(-1).fillna(machine.now_dt.datetime)
    if df.empty:
        return

    fig = px.line(df, x='ds', y='value', color='obj', facet_row='key',
                  title='Simulation - Numerical', **kwargs)
    fig.update_yaxes(matches=None)
    fig.for_each_annotation(lambda a: a.update(text=a.text.replace('key=', '')))
    fig.show()


machine = list(machines.values())[0]


plot_timeline(
    operator.data['categorical']
    + machine.data['categorical']
    # + sum([list(p.bom.values())[0]['consumable'].data['categorical']
    #        for p in machine.programs.values()], [])
    + machine.schedule.data['categorical']
, width=800, height=1000)

# plot_numerical(
#     machine.data['numerical']
#     + operator.data['numerical']
#     + sum([list(p.bom.values())[0]['consumable'].data['numerical']
#            for p in machine.programs.values()], [])
# ,width=800, height=1400)

In [ ]:
%load_ext autoreload
%autoreload 2

# Config
import logging
import sys

import arrow
import matplotlib.pyplot as plt
import pandas as pd
import simpy

from src.machine import *
from src.operator import *
from src.utils import *
from src.base import *


logging.basicConfig(
     stream=sys.stdout,
     level=logging.DEBUG,
     format='%(asctime)s - %(name)s - %(levelname)-7s - %(message)s',
     datefmt='%H:%M:%S'
 )
logger = logging.getLogger(__name__)
logger.info('Starting simulation')


# Simulation
start = arrow.now('Europe/Helsinki')
env = simpy.Environment(initial_time=start.timestamp())
machine = Machine(env)
operator = Operator(env).assign_machine(machine)

env.run(until=start.timestamp() + 7 * 24 * 60 * 60)

Missing:
- Planned maintenance
- Material / batch ids (materiaali-id voi olla string)
- Collectng events (debug / info / warning / error) --> logs not for now, separate variable when necessary, error/stop code most critical for now (integer) --> exact variables in Excel
- Mappings from string states into integers / boolean
- Randomization of fails - "global fail proba based on last maintenance"
- Integration: Pull/push? Taking snapshots of the machine state vs. pushing e.g. error
- Cumulative: Consumable
- Program: add part counter / target / postprocessing step that identifies failed  
- Randomization of time


In [ ]:
# 19.12. palaveri